In [89]:
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from selectolax.parser import HTMLParser
import mailparser
import pickle
import base64
import email
import datetime
import pandas as pd
import traceback
import qgrid


class WalnutDataSaver():
    filter_query="from:report-no-reply@getwalnut.in"
    walnut_payment_keywords_to_source ={"hdfc":"hdfc",
                                    "icici":"icici",
                                    "kotak":"kotak",
                                        "paytm":"paytm",
                                        "amex":"amex"}
    @staticmethod
    def clean_walnut_csv_and_save_df(mail_utils_class, datastore_path, sheet_id):
        if not WalnutDataSaver.save_walnut_attachment(mail_utils_class, datastore_path):
            print("no walnut attachment")
            return
        with open(datastore_path+'/walnut_expenses.csv','rb') as f:
            walnut_df=pd.read_csv(f)

        # Changing column names
        walnut_df.columns=walnut_df[4:5].values.tolist()[0]
        
        #Removing useless rows
        walnut_df=walnut_df[5:-1]
        
        walnut_df=walnut_df.rename(columns={"TIME":"time","AMOUNT":"amount"})
        walnut_df['from']=walnut_df['ACCOUNT']
        walnut_df['time']=walnut_df['DATE']+" "+walnut_df['time']
        walnut_df['time'] = pd.to_datetime(walnut_df['time'], format="%d-%m-%y %I:%M %p")

        walnut_df['time']=walnut_df.time.map(lambda x: x.replace(second=0,microsecond=0))
        walnut_df['amount']=walnut_df.amount.map(lambda x: x.replace(",",""))
        walnut_df['amount'] = walnut_df['amount'].astype('float') 

        # Replace all elements of from column from which contains keys of walnut_pay.. dict with corresponding values
        # this causes hdfc credit card to be replaced with hdfc, but does not cause an issue right now
        # because we later pickup only transactions which we have record of from mail
        for key,value in WalnutDataSaver.walnut_payment_keywords_to_source.items():
            walnut_df.loc[walnut_df['from'].str.contains(key,case=False), 'from'] = value
        walnut_df.to_pickle(datastore_path+'/walnut_processed.pickle')
        walnut_df['time']=walnut_df.time.map(lambda x: x.timestamp())
        walnut_df=walnut_df.fillna("")
        mail_utils_class.write_data_to_google_sheet(sheet_id,"walnut",walnut_df.T.reset_index().T.values.tolist())
        
        
    @staticmethod
    def save_walnut_attachment(mail_utils_class, datastore_path):
        walnut_msg_ids=mail_utils_class.get_all_message_ids(WalnutDataSaver.filter_query)
        # Assuming that gmail always serves messages by reverse time sorting, so that we get latest walnut email
        # available
        try:
            walnut_msg_id = walnut_msg_ids[0]['id']
        except IndexError:
            return False
        message=mail_utils_class.fetch_mail_from_id(walnut_msg_id,raw_format=False)
        found_csv=False
        for part in message['payload']['parts']:
            if part['filename'] and ".csv" in part['filename']:
                attachment_id = part['body']['attachmentId']
                attachment= mail_utils_class.gmail_service.users().messages().attachments().get(userId='me',
                                                                                 messageId=walnut_msg_id,
                                                                                 id=attachment_id).execute()
                file_data = base64.urlsafe_b64decode(attachment['data'].encode('UTF-8'))
                path = '/'.join([datastore_path, 'walnut_expenses.csv'])
                f = open(path, 'wb')
                f.write(file_data)
                f.close()
                found_csv=True
                break
        if not found_csv:
            print("No attachment was found in latest walnut mail.")
            return False
        else:
            return True
                
# If modifying these scopes, delete the file token.pickle.
class MailUtils():
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly','https://www.googleapis.com/auth/spreadsheets']
    def __init__(self, datastore_path):        
        if not os.path.exists(datastore_path):
            print(f"Creating Directory {datastore_path}")
            os.mkdir(datastore_path)
        self.token_path = datastore_path+"/token.pickle"
        self.client_secrets_path = datastore_path+"/credentials.json" #'/Users/sanchitsharma/Documents/spentAnalysis/credentials.json'
        self.gmail_service = self.get_google_service(service_name='gmail') 
    
    @classmethod
    def get_text_from_mail_body(cls,html):
        tree = HTMLParser(html)

        if tree.body is None:
            return None

        for tag in tree.css('script'):
            tag.decompose()
        for tag in tree.css('style'):
            tag.decompose()

        text = tree.body.text(separator='\n')
        return text
    
    def get_google_service(self, service_name="gmail", version='v1'): 
        # The file token.pickle stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first
        # time.
        creds = None        
        if os.path.exists(self.token_path):
            with open(self.token_path, 'rb') as token:
                creds = pickle.load(token)
                
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                try:
                    flow = InstalledAppFlow.from_client_secrets_file(self.client_secrets_path, MailUtils.SCOPES)
                    creds = flow.run_local_server(port=0)
                except FileNotFoundError:
                    raise ValueError(f"Download credentials file in {datastore_path} folder by enabling the GMAIL API for your account at - https://developers.google.com/gmail/api/quickstart/python")
            # Save the credentials for the next run
            with open(self.token_path, 'wb') as token:
                pickle.dump(creds, token)
        return build(service_name, version, credentials=creds)
    
    def get_all_message_ids(self,filter_query):
    
        #some_days_ago_date_str=(datetime.datetime.now() - datetime.timedelta(days=5)).strftime("%d-%m-%Y")

        #query=f'{filter_query},after:{some_days_ago_date_str}'

        tomorrow_date_str = (datetime.datetime.now() + datetime.timedelta(days=1)).strftime("%m-%d-%Y")
        query = f'{filter_query},before:{tomorrow_date_str}'
        print(query)
        response = self.gmail_service.users().messages().list(userId='me', q=query).execute()
        message_ids = response.get('messages', [])

        while 'nextPageToken' in response:
            page_token = response['nextPageToken']
            response = self.gmail_service.users().messages().list(userId='me', q=query,
                                                       pageToken=page_token).execute()
            message_ids.extend(response['messages'])
        return message_ids
    
    def fetch_mail_from_id(self,msg_id,raw_format=True):
        """    
        :param gmail_service: 
        :param id: gmail id of the mail 
        :return: mail object
        """
        if raw_format:            
            return self.gmail_service.users().messages().get(userId='me', id=msg_id, format='raw').execute()
        else:
            return self.gmail_service.users().messages().get(userId='me', id=msg_id).execute()
    
    def fetch_mail_details(self,message_id):
        message = self.fetch_mail_from_id(message_id)         
        try:
            msg_str = base64.urlsafe_b64decode(message['raw'].encode('ASCII')).decode('utf-8')
            mail_body = mailparser.parse_from_string(msg_str).body
            text_body = MailUtils.get_text_from_mail_body(mail_body)
            mail_details = {"mail_text":text_body.split(),'timestamp':int(message['internalDate'])/1000}
            return mail_details
        except UnicodeDecodeError:
            print(f"Error message on f{datetime.datetime.fromtimestamp(int(message['internalDate'])/1000)} for message_id f{message_id}")
    
    def read_data_from_google_sheet(self, sheet_id, subsheet_name):
        sheet_service = self.get_google_service(service_name="sheets",version='v4')
        result_input = sheet_service.spreadsheets().values().get(spreadsheetId=sheet_id,
                                    range=subsheet_name+"!A1:AA10000").execute()
        values_input = result_input.get('values', [])                  
        return values_input
                  
    def write_data_to_google_sheet(self, sheet_id, subsheet_name, rows_to_be_written):
        sheet_service = self.get_google_service(service_name="sheets",version='v4')
        creat_sheet_body = {"requests": [
                                    {"addSheet": {
                                            "properties": {"title": subsheet_name}
                                      }}            
                                      ]}
        try:
            sheet_service.spreadsheets().batchUpdate(body=creat_sheet_body,
                                            spreadsheetId=sheet_id).execute()
        except:
            pass
        sheet_service.spreadsheets().values().update(
        spreadsheetId=sheet_id,
        valueInputOption='RAW',
        range=subsheet_name+"!A1:AA10000",
        body=dict(
            majorDimension='ROWS',
            values=rows_to_be_written)
        ).execute()
        
class TransactionMailManager():
                  
    payment_source_to_filter_dict = {'paytm':"from:no-reply@paytm.com",
                                     "kotak":"from:bankalerts@kotak.com",
                                     "hdfc":"from:alerts@hdfcbank.net",
                                     "amex":"americanexpress@welcome.aexp.com",
                                     "phonepe":"noreply@phonepe.com",
                                     "mobikwik":"no-reply@mobikwik.com",
                                    "walnut":"placeholder"}

                  
    def __init__(self, username, payment_source_name=None, write_to_sheet_id=None):
        self.username=username
        self.payment_source_name = payment_source_name
        self.write_to_sheet_id = write_to_sheet_id
        self.datastore_path = username+"_datastore"
        self.filter_query = self.__class__.payment_source_to_filter_dict[payment_source_name]
        self.messages_store_path = self.datastore_path +"/"+ payment_source_name+".pickle"
        self.mail_utils = MailUtils(self.datastore_path)
    
    
    def get_old_messages_dict_from_sheet(self):    
        list_values = self.mail_utils.read_data_from_google_sheet(self.write_to_sheet_id,self.payment_source_name)        
        return {row[0]:{'mail_text':row[1].split(),'timestamp':int(row[2])} for row in list_values}
                  
    def get_old_messages_dict(self):                  
        try:
            with open(self.messages_store_path,'rb') as fl:
                old_messages = pickle.load(fl)
            print(f"Got  {len(old_messages)} old messages.")
        except IOError as e:
            print(f"Got IOError - {e}")
            old_messages = {} 
                  
        return old_messages
    
    def mail_fetcher(self):  
        """
        This will store all mails from this filter_query in a file of the same name.
        Mails will be pickled in following format 
        {<mail_id>:{'mail_text':<mail text (each word separated by *;*)>, 'datetime':<sent time of mail>)
        If a file with same name exists, it will only fetch those messages which are not already contained in the file.  
        :param filter_query: filter to pass gmail api to search mails 
        :param to_store_name: name to store the file with 
        :return: None
        """
        old_messages = self.get_old_messages_dict()
        old_messages_from_sheet = self.get_old_messages_dict_from_sheet()
#         correct=True
        for key,value in old_messages_from_sheet.items():
            if not value==old_messages[key]:
                print("Problem",value,"\nProblem",old_messages[key])
                break
        message_ids = self.mail_utils.get_all_message_ids(self.filter_query)
        new_messages_dict = {}
        msg_ids_not_in_file=[msg_id_dict for msg_id_dict in message_ids if msg_id_dict['id'] not in old_messages]

        for i, msg_id_dict in enumerate(msg_ids_not_in_file):
            msg_id = msg_id_dict['id']
#             print("getting msg for",msg_id)
            new_messages_dict[msg_id] = self.mail_utils.fetch_mail_details(msg_id)

        old_messages.update(new_messages_dict)
        print(f"Final number of messages {len(old_messages)}.\n-----------------\n")
            
        with open(self.messages_store_path,'wb') as fl:  
            pickle.dump(old_messages,fl,protocol=pickle.HIGHEST_PROTOCOL)
#         print(list(old_messages.items())[0])
        if self.write_to_sheet_id:
            list_to_be_written = [[key," ".join(dic['mail_text']),dic['timestamp']] for key,dic in old_messages.items() if dic is not None]
            list_to_be_written.sort(key=lambda row:row[2])                  
            self.mail_utils.write_data_to_google_sheet(self.write_to_sheet_id,self.payment_source_name,
                                                       list_to_be_written)
        
                  
    def execute(self):                  
        if self.payment_source_name=='walnut':
            WalnutDataSaver.clean_walnut_csv_and_save_df(self.mail_utils, self.datastore_path, self.write_to_sheet_id)
        else:
            
            self.mail_fetcher()

def store_transactions_messages_in_datastore(username,payment_sources,write_to_sheet_id):
    for payment_source_name in payment_sources:
        tmm=TransactionMailManager(username,payment_source_name=payment_source_name,write_to_sheet_id=write_to_sheet_id)
        tmm.execute()

In [91]:
#####Sulinder
# username="sulinder"
# payment_sources=["kotak","paytm","phonepe","mobikwik"]
# sheet_id="1Hyk2Ex7xN96lisiNnNtRxQ5nQeMcabml1Rdza2Cjoc8"
#####Sanchit
username="sanchit"
payment_sources=["kotak","hdfc","paytm","walnut"]
# payment_sources=["walnut"]
sheet_id="1gQoZfriTs7pXRJFEdbyCOe0-NsPzIwKbCenVEZVrphA"
######
store_transactions_messages_in_datastore(username,payment_sources,sheet_id)

Got  247 old messages.
from:bankalerts@kotak.com,before:05-05-2020
Final number of messages 247.
-----------------

Got  151 old messages.
from:alerts@hdfcbank.net,before:05-05-2020
Final number of messages 151.
-----------------

Got  1212 old messages.
from:no-reply@paytm.com,before:05-05-2020
Final number of messages 1212.
-----------------

from:report-no-reply@getwalnut.in,before:05-05-2020


In [8]:
qgrid.show_grid(pd.read_pickle("sanchit_datastore/walnut_processed.pickle"))

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [3]:
s=[3,4,5,1]
s.sort()
s

[1, 3, 4, 5]

In [63]:
pd.DataFrame([1,2,3,None])

,0
0,1.0
1,2.0
2,3.0
3,NaN


In [71]:
{"a":{2:"a",1:"e32e3"}}=={"a":{2:"a",1:"e32e3"}}

True